In [1]:
import pandas as pd
import numpy as np
from khaiii import KhaiiiApi
from gensim import corpora, models
from pprint import pprint


In [2]:
year = [i for i in range(1997, 2021)]
sortedresult = pd.DataFrame()

In [3]:
for i in range(24):
    print("==== "+str(year[i])+" ====")
    data = pd.read_csv("./modi_data/kor_"+str(year[i])+".csv")
    #data = data.drop(['Unnamed: 0'], axis=1)
    
    dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
    dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
    #print(dfWordList2)

    dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
    dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
    dfWordDiv = dfWordList2
    #print(dfWordMod)

    seriesDelete = dfWordDel["기존"]
    stopword = []
    for word in seriesDelete.values:
        stopword.append(word)
    #print(stopword)

    seriesModify = dfWordMod["기존"]
    modiword = []
    for word in seriesModify.values:
        modiword.append(word)
    #print(len(modiword))

    seriesModify2 = dfWordMod["수정"]
    modiword2 = []
    for word in seriesModify2.values:
        modiword2.append(word)
    #print(len(modiword2))

    seriesDivide = dfWordDiv["기존"]
    divword = []
    for word in seriesDivide.values:
        divword.append(word)
    #print(divword)
    #print(len(divword))

    seriesDivide2 = dfWordDiv["수정"]
    divword2 = []
    for words in seriesDivide2.values:
        divword2.append(words.split(', '))
    #print(divword2)
    #print(len(divword2))


    api = KhaiiiApi()
    def khaiiiTokenizer(raw, stopword=stopword, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']): # 일반명사 고유명사 의존명사 대명사 수사 외국어
        list = []
        skip = 0

        for word in api.analyze(raw): #raw data
            #print(word)
            
            for i, morph in enumerate(word.morphs):
                #print(morph.lex)
                if skip == 1: 
                    #print(morph.lex) # '지능'
                    skip = 0
                    continue

                if morph.lex == '인공' and i+1 < len(word.morphs) and word.morphs[i+1].lex == "지능":
                    #print(morph.lex + word.morphs[i+1].lex) # 인공지능
                    list.append(morph.lex + word.morphs[i+1].lex)
                    skip = 1
                    continue

                if len(morph.lex) > 1 and morph.tag in pos and morph.lex not in stopword: 
                    if morph.tag == 'SL':
                        morph.lex = morph.lex.lower()
                    if morph.lex in divword:
                        morph.lex = divword2[divword.index(morph.lex)]
                        list.extend(morph.lex)
                    elif morph.lex in modiword:
                        morph.lex = modiword2[modiword.index(morph.lex)]
                        list.append(morph.lex)
                    else: list.append(morph.lex)
                    
        return list

    tokenized = data["kor_full"].apply(lambda row: khaiiiTokenizer(row))
    #print(tokenized)
    #tokenized.to_excel("./finaldata/0911token_full.xls") ##############
    print("========= tokenization completed =========")

    id2word = corpora.Dictionary(tokenized)
    corpus=[id2word.doc2bow(text) for text in tokenized]
    print("# words in total : ", len(id2word))
    print("# documents : ", len(corpus))

    #tfidf
    print("==== calculating tfidf ====")
    tfidf = models.TfidfModel(corpus)

    #tfidf per doc
    tfidflist = []
    for doc in tfidf[corpus]:
        inner_list = [0]*len(id2word) 
        for id, freq in doc:
            inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
        tfidflist.append(inner_list)
    #print(len(tfidflist))

    tfidf_df = pd.DataFrame(tfidflist)
    tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
    #print(tfidf_df)

    total_df = pd.concat([data[["year", "no"]], tfidf_df], axis=1)
    #print(total_df)
    total_df.to_excel("./final_data/tfidf"+str(year[i])+".xlsx") 

    #sum of tfidf for each word
    columnsum = pd.DataFrame(total_df.sum(axis=0)).T
    columnsum = columnsum.drop(['no'], axis=1)
    columnsum['year'] = year[i]
    #print(columnsum)
    columnsum.to_excel("./final_data/sum"+str(year[i])+".xlsx")

    #sort tfidf value in descending order
    columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
    print(columnsum)
    columnsum.to_excel("./final_data/sorted"+str(year[i])+".xlsx")

    print("==== completed ====")

==== 1997 ====
========= tokenization completed =========
# words in total :  71
# documents :  10
==== calculating tfidf ====
   year    교육    연구   cai    자료  web    개별  멀티미디어    원격   편집기  ...    자기  \
0  1997  1.02  0.82  0.81  0.78  0.7  0.68   0.66  0.66  0.66  ...  0.11   

    서비스    일선    원인    연결   안내자    실정   시스템    다양    학생  
0  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  

[1 rows x 72 columns]
==== completed ====
==== 1998 ====
========= tokenization completed =========
# words in total :  457
# documents :  34
==== calculating tfidf ====
   year    교육    정보  에이전트   시스템    설계    연구   컴퓨터  소프트웨어    초등  ...   베이스  \
0  1998  2.24  1.72   1.6  1.58  1.37  1.33  1.29   1.24  1.22  ...  0.06   

     선정  시뮬레이션  시물레이션  dijkstra  modified    도로   디지털    소요    실용  
0  0.06   0.06   0.06      0.06      0.06  0.06  0.06  0.06  0.06  

[1 rows x 458 columns]
==== completed ====
==== 1999 ====
========= tokenization completed =========
# words in total :  484
# documents :  2

In [4]:
# tfidf for full data

data = pd.read_csv("./modi_data/kor_full.csv")
#data = data.drop(['Unnamed: 0'], axis=1)
    
dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
#print(dfWordList2)

dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
dfWordDiv = dfWordList2
#print(dfWordMod)

seriesDelete = dfWordDel["기존"]
stopword = []
for word in seriesDelete.values:
    stopword.append(word)
#print(stopword)

seriesModify = dfWordMod["기존"]
modiword = []
for word in seriesModify.values:
    modiword.append(word)
#print(len(modiword))

seriesModify2 = dfWordMod["수정"]
modiword2 = []
for word in seriesModify2.values:
    modiword2.append(word)
#print(len(modiword2))

seriesDivide = dfWordDiv["기존"]
divword = []
for word in seriesDivide.values:
    divword.append(word)
#print(divword)
#print(len(divword))

seriesDivide2 = dfWordDiv["수정"]
divword2 = []
for words in seriesDivide2.values:
    divword2.append(words.split(', '))
#print(divword2)
#print(len(divword2))


api = KhaiiiApi()
def khaiiiTokenizer(raw, stopword=stopword, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']): # 일반명사 고유명사 의존명사 대명사 수사 외국어
    list = []
    skip = 0

    for word in api.analyze(raw): #raw data
        #print(word)
        
        for i, morph in enumerate(word.morphs):
            #print(morph.lex)
            if skip == 1: 
                #print(morph.lex) # '지능'
                skip = 0
                continue

            if morph.lex == '인공' and i+1 < len(word.morphs) and word.morphs[i+1].lex == "지능":
                #print(morph.lex + word.morphs[i+1].lex) # 인공지능
                list.append(morph.lex + word.morphs[i+1].lex)
                skip = 1
                continue

            if len(morph.lex) > 1 and morph.tag in pos and morph.lex not in stopword: 
                if morph.tag == 'SL':
                    morph.lex = morph.lex.lower()
                if morph.lex in divword:
                    morph.lex = divword2[divword.index(morph.lex)]
                    list.extend(morph.lex)
                elif morph.lex in modiword:
                    morph.lex = modiword2[modiword.index(morph.lex)]
                    list.append(morph.lex)
                else: list.append(morph.lex)
                
    return list

tokenized = data["kor_full"].apply(lambda row: khaiiiTokenizer(row))
#print(tokenized)
#tokenized.to_excel("./finaldata/0911token_full.xls") ##############
print("========= tokenization completed =========")

id2word = corpora.Dictionary(tokenized)
corpus=[id2word.doc2bow(text) for text in tokenized]
print("# words in total : ", len(id2word))
print("# documents : ", len(corpus))

#tfidf
print("==== calculating tfidf ====")
tfidf = models.TfidfModel(corpus)

#tfidf per doc
tfidflist = []
for doc in tfidf[corpus]:
    inner_list = [0]*len(id2word) 
    for id, freq in doc:
        inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
    tfidflist.append(inner_list)
#print(len(tfidflist))

tfidf_df = pd.DataFrame(tfidflist)
tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
#print(tfidf_df)

total_df = pd.concat([data[["year", "no"]], tfidf_df], axis=1)
#print(total_df)
total_df.to_excel("./final_data/tfidf_full.xlsx") 

#sum of tfidf for each word
columnsum = pd.DataFrame(total_df.sum(axis=0)).T
columnsum = columnsum.drop(['no'], axis=1)
columnsum['year'] = year[i]
#print(columnsum)
columnsum.to_excel("./final_data/sum_full.xlsx")

#sort tfidf value in descending order
columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
print(columnsum)
columnsum.to_excel("./final_data/sorted_full.xlsx")

print("==== completed ====")

========= tokenization completed =========
# words in total :  5086
# documents :  1149
==== calculating tfidf ====
   year     학습  프로그래밍     정보     로봇    컴퓨터    시스템     교육     sw     평가  ...  \
0  2020  38.82  36.43  36.23  33.03  31.51  28.13  26.34  25.67  25.39  ...   

    양육자    소득    가계    영유    오전    오후    장르    주말    주중    영아  
0  0.04  0.04  0.04  0.04  0.04  0.04  0.04  0.04  0.04  0.04  

[1 rows x 5087 columns]
==== completed ====
